In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('./train/train.csv')
sub = pd.read_csv('./sample_submission.csv')

In [3]:
#테스트 로드
for i in range(0,81):
    s1 = "test_%d = pd.read_csv('./test/%d.csv')"%(i,i)
    exec(s1)
    
test = pd.DataFrame()
for i in range(0,81):
    s1 = "test = pd.concat([test, test_%d], axis = 0)"%(i)
    exec(s1)

In [4]:
train.corr()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
Day,1.000000,0.000000,0.000000,-0.027802,0.021901,0.038477,-0.127688,0.072897,-0.002505
Hour,0.000000,1.000000,0.000000,0.029905,-0.018094,-0.055903,-0.084275,0.105528,0.003817
Minute,0.000000,0.000000,1.000000,-0.001305,-0.000997,-0.009522,0.008839,-0.007646,-0.000240
DHI,-0.027802,0.029905,-0.001305,1.000000,0.288294,0.203286,-0.478503,0.457813,0.666908
DNI,0.021901,-0.018094,-0.000997,0.288294,1.000000,0.219555,-0.611184,0.402460,0.833547
WS,0.038477,-0.055903,-0.009522,0.203286,0.219555,1.000000,-0.230035,0.027693,0.238521
RH,-0.127688,-0.084275,0.008839,-0.478503,-0.611184,-0.230035,1.000000,-0.532777,-0.677178
T,0.072897,0.105528,-0.007646,0.457813,0.402460,0.027693,-0.532777,1.000000,0.561990
TARGET,-0.002505,0.003817,-0.000240,0.666908,0.833547,0.238521,-0.677178,0.561990,1.000000


In [5]:
#타겟변수
train_day_1 = train.loc[train['Day'] > 0].reset_index(drop = True)
train_day_2 = train.loc[train['Day'] > 1].reset_index(drop = True)

In [6]:
#train_1 : 1일 뒤를 예측할 모델, train_2 : 2일 뒤를 예측할 모델
train_1 = pd.concat([train, train_day_1.rename(columns = {'TARGET' : 'TARGET_1'})['TARGET_1']], axis = 1).dropna(axis = 0)
train_2 = pd.concat([train, train_day_2.rename(columns = {'TARGET' : 'TARGET_2'})['TARGET_2']], axis = 1).dropna(axis = 0)

In [7]:
#7일간격으로 Day변경
day_list_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8]
for j in day_list_2: 
    for k in range(0,2025,7):
        train_1['Day'].loc[train_1['Day'] == j+k] = j
        train_2['Day'].loc[train_2['Day'] == j+k] = j

C:\Users\Kim\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
test_1 = test.copy()
test_2 = test.copy()

TARGET의 흐름과 유사한 데이터만 뽑아서 학습

In [14]:
test_1 = test_1.loc[test_1['Day'] == 6].reset_index(drop = True)
test_2 = test_2.loc[test_2['Day'] == 6].reset_index(drop = True)

In [15]:
x_train_1 = train_1.drop(['Day', 'Minute', 'TARGET_1'], axis = 1)
x_train_2 = train_2.drop(['Day', 'Minute', 'TARGET_2'], axis = 1)

y_train_1 = train_1['TARGET_1']
y_train_2 = train_2['TARGET_2']

x_test_1 = test_1.drop(['Day', 'Minute'], axis = 1)
x_test_2 = test_2.drop(['Day', 'Minute'], axis = 1)

In [16]:
#TARGET의 형태가 비슷한 범주끼리 0으로 처리
num_list = [1, 2, 3, 4, 5, 20, 21, 22, 23]
for i in num_list:
    x_train_1['Hour'].loc[x_train_1['Hour'] == i] = 0
    x_train_2['Hour'].loc[x_train_2['Hour'] == i] = 0
    x_test_1['Hour'].loc[x_test_1['Hour'] == i] = 0
    x_test_2['Hour'].loc[x_test_2['Hour'] == i] = 0

C:\Users\Kim\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Hour, DHI, DNI, WS, RH, T, TARGET만 넣고 돌렸을 때 (Hour만 전처리) = 1.92

In [20]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [22]:
#1일 뒤를 예측하는 모델
quantiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
pred_1 = pd.DataFrame()
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
for n_fold, (train_idx, val_idx) in enumerate(folds.split(x_train_1)):
    train_x, train_y = x_train_1.loc[train_idx], y_train_1.loc[train_idx]
    val_x, val_y = x_train_1.loc[val_idx], y_train_1.loc[val_idx]
    print(n_fold)
    for i in quantiles:
        print(i)
        model = LGBMRegressor(objective = 'quantile', boosting = 'gbdt', n_estimators = 10000, learning_rate = 0.009,
                              colsample_bytree= 0.8, seed = 2020, max_depth = 3, num_leaves = 24, alpha = i)
        model.fit(train_x, train_y, eval_metric = ['quantile'], eval_set = [(train_x, train_y), (val_x, val_y)], early_stopping_rounds = 200, verbose = 500)
        pred_1 = pd.concat([pred_1, pd.DataFrame(model.predict(x_test_1))], axis = 1)


result_1 = pred_1.iloc[:,0:9]
result_2 = pred_1.iloc[:,9:18]
result_3 = pred_1.iloc[:,18:27]
result_4 = pred_1.iloc[:,27:36]
result_5 = pred_1.iloc[:,36:45]

result = (result_1+result_2+result_3+result_4+result_5)/5
result.columns = sub.iloc[:,1:].columns

result

0
0.1
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 1.40733	valid_1's quantile: 1.3862
[1000]	training's quantile: 1.39286	valid_1's quantile: 1.37484
[1500]	training's quantile: 1.3821	valid_1's quantile: 1.36711
[2000]	training's quantile: 1.37135	valid_1's quantile: 1.3605
[2500]	training's quantile: 1.36614	valid_1's quantile: 1.35765
[3000]	training's quantile: 1.35956	valid_1's quantile: 1.35315
[3500]	training's quantile: 1.35567	valid_1's quantile: 1.34928
[4000]	training's quantile: 1.35121	valid_1's quantile: 1.34793
[4500]	training's quantile: 1.3476	valid_1's quantile: 1.34577
[5000]	training's quantile: 1.34424	valid_1's quantile: 1.34461
[5500]	training's quantile: 1.34335	valid_1's quantile: 1.34416
[6000]	training's quantile: 1.34061	valid_1's quantile: 1.34353
[6500]	training's quantile: 1.33848	valid_1's quantile: 1.34301
[7000]	training's quantile: 1.33592	valid_1's quantile: 1.34265
[7500]	training's quantile: 1.33333	valid

0.7
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.22451	valid_1's quantile: 2.22858
[1000]	training's quantile: 2.08163	valid_1's quantile: 2.0898
[1500]	training's quantile: 2.04838	valid_1's quantile: 2.05621
[2000]	training's quantile: 2.02721	valid_1's quantile: 2.03413
[2500]	training's quantile: 2.0143	valid_1's quantile: 2.02213
[3000]	training's quantile: 2.00292	valid_1's quantile: 2.01138
[3500]	training's quantile: 1.99274	valid_1's quantile: 2.00229
[4000]	training's quantile: 1.98429	valid_1's quantile: 1.99454
[4500]	training's quantile: 1.97948	valid_1's quantile: 1.99099
[5000]	training's quantile: 1.97408	valid_1's quantile: 1.98676
[5500]	training's quantile: 1.96918	valid_1's quantile: 1.98371
[6000]	training's quantile: 1.96411	valid_1's quantile: 1.97979
[6500]	training's quantile: 1.95993	valid_1's quantile: 1.977

[5000]	training's quantile: 2.59089	valid_1's quantile: 2.57871
[5500]	training's quantile: 2.58585	valid_1's quantile: 2.57835
Early stopping, best iteration is:
[5316]	training's quantile: 2.58757	valid_1's quantile: 2.57778
0.4
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.90008	valid_1's quantile: 2.84817
[1000]	training's quantile: 2.82711	valid_1's quantile: 2.77944
[1500]	training's quantile: 2.80069	valid_1's quantile: 2.75745
[2000]	training's quantile: 2.78333	valid_1's quantile: 2.74486
[2500]	training's quantile: 2.77043	valid_1's quantile: 2.73648
[3000]	training's quantile: 2.75782	valid_1's quantile: 2.73134
[3500]	training's quantile: 2.74823	valid_1's quantile: 2.72602
[4000]	training's quantile: 2.74163	valid_1's quantile: 2.72179
[4500]	training's quantile: 2.73507	valid_1's quantile: 2.71913
[5000]	training's quanti

[6000]	training's quantile: 0.883521	valid_1's quantile: 0.898894
[6500]	training's quantile: 0.88136	valid_1's quantile: 0.897147
[7000]	training's quantile: 0.879221	valid_1's quantile: 0.894945
[7500]	training's quantile: 0.878003	valid_1's quantile: 0.893791
[8000]	training's quantile: 0.876852	valid_1's quantile: 0.892581
[8500]	training's quantile: 0.875698	valid_1's quantile: 0.89147
[9000]	training's quantile: 0.874785	valid_1's quantile: 0.89063
[9500]	training's quantile: 0.873759	valid_1's quantile: 0.889655
[10000]	training's quantile: 0.872797	valid_1's quantile: 0.888829
Did not meet early stopping. Best iteration is:
[10000]	training's quantile: 0.872797	valid_1's quantile: 0.888829
2
0.1
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 1.39044	valid_1's quantile: 1.45407
[1000]	training's quantile: 1.37464	valid_1's quantile

[2500]	training's quantile: 2.00791	valid_1's quantile: 2.0419
[3000]	training's quantile: 1.99689	valid_1's quantile: 2.03205
[3500]	training's quantile: 1.98669	valid_1's quantile: 2.02318
[4000]	training's quantile: 1.9789	valid_1's quantile: 2.01707
[4500]	training's quantile: 1.97145	valid_1's quantile: 2.01072
[5000]	training's quantile: 1.96498	valid_1's quantile: 2.00495
[5500]	training's quantile: 1.95865	valid_1's quantile: 1.9997
[6000]	training's quantile: 1.95304	valid_1's quantile: 1.9954
[6500]	training's quantile: 1.94886	valid_1's quantile: 1.99224
[7000]	training's quantile: 1.94537	valid_1's quantile: 1.99012
[7500]	training's quantile: 1.9416	valid_1's quantile: 1.988
[8000]	training's quantile: 1.93855	valid_1's quantile: 1.98599
[8500]	training's quantile: 1.93619	valid_1's quantile: 1.98452
[9000]	training's quantile: 1.93357	valid_1's quantile: 1.98276
[9500]	training's quantile: 1.93152	valid_1's quantile: 1.98162
[10000]	training's quantile: 1.92985	valid_1's 

[7000]	training's quantile: 2.68786	valid_1's quantile: 2.76871
[7500]	training's quantile: 2.68368	valid_1's quantile: 2.76795
Early stopping, best iteration is:
[7542]	training's quantile: 2.6832	valid_1's quantile: 2.7679
0.5
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.84133	valid_1's quantile: 2.83718
[1000]	training's quantile: 2.73263	valid_1's quantile: 2.74503
[1500]	training's quantile: 2.7001	valid_1's quantile: 2.72147
[2000]	training's quantile: 2.68117	valid_1's quantile: 2.70721
[2500]	training's quantile: 2.66792	valid_1's quantile: 2.69993
[3000]	training's quantile: 2.65726	valid_1's quantile: 2.69605
[3500]	training's quantile: 2.64933	valid_1's quantile: 2.6915
[4000]	training's quantile: 2.64168	valid_1's quantile: 2.68793
[4500]	training's quantile: 2.63513	valid_1's quantile: 2.68632
[5000]	training's quantile: 

[2000]	training's quantile: 1.36212	valid_1's quantile: 1.41419
[2500]	training's quantile: 1.35522	valid_1's quantile: 1.41094
[3000]	training's quantile: 1.34922	valid_1's quantile: 1.40611
[3500]	training's quantile: 1.34457	valid_1's quantile: 1.40434
[4000]	training's quantile: 1.34301	valid_1's quantile: 1.40368
[4500]	training's quantile: 1.33961	valid_1's quantile: 1.40171
[5000]	training's quantile: 1.33736	valid_1's quantile: 1.40076
Early stopping, best iteration is:
[5051]	training's quantile: 1.3372	valid_1's quantile: 1.40074
0.2
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.24977	valid_1's quantile: 2.32329
[1000]	training's quantile: 2.21686	valid_1's quantile: 2.29326
[1500]	training's quantile: 2.20599	valid_1's quantile: 2.28706
[2000]	training's quantile: 2.19252	valid_1's quantile: 2.27763
[2500]	training's quantil

0.7
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.20726	valid_1's quantile: 2.28758
[1000]	training's quantile: 2.07167	valid_1's quantile: 2.14555
[1500]	training's quantile: 2.04242	valid_1's quantile: 2.11632
[2000]	training's quantile: 2.02087	valid_1's quantile: 2.09433
[2500]	training's quantile: 2.00679	valid_1's quantile: 2.08029
[3000]	training's quantile: 1.99703	valid_1's quantile: 2.07126
[3500]	training's quantile: 1.98501	valid_1's quantile: 2.05968
[4000]	training's quantile: 1.97728	valid_1's quantile: 2.0523
[4500]	training's quantile: 1.97151	valid_1's quantile: 2.04715
[5000]	training's quantile: 1.96668	valid_1's quantile: 2.04274
[5500]	training's quantile: 1.96159	valid_1's quantile: 2.03768
[6000]	training's quantile: 1.95671	valid_1's quantile: 2.03282
[6500]	training's quantile: 1.95248	valid_1's quantile: 2.02

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
1,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
2,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
3,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
4,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
3884,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
3885,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33
3886,0.0,0.0,0.0,0.0,0.0,2.778748e-26,1.105514e-33,1.106803e-33,1.105470e-33


In [402]:
sub.loc[sub.id.str.contains("Day7"), "q_0.1":] = result.sort_index().values

In [23]:
#2일 뒤를 예측하는 모델
quantiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
pred_2 = pd.DataFrame()
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
for n_fold, (train_idx, val_idx) in enumerate(folds.split(x_train_2)):
    train_x, train_y = x_train_2.loc[train_idx], y_train_2.loc[train_idx]
    val_x, val_y = x_train_2.loc[val_idx], y_train_2.loc[val_idx]
    print(n_fold)
    for i in quantiles:
        print(i)
        model = LGBMRegressor(objective = 'quantile', boosting = 'gbdt', n_estimators = 10000, learning_rate = 0.009,
                              colsample_bytree= 0.8, seed = 2020, max_depth = 3, num_leaves = 24, alpha = i)
        model.fit(train_x, train_y, eval_metric = ['quantile'], eval_set = [(train_x, train_y), (val_x, val_y)], early_stopping_rounds = 200, verbose = 500)
        pred_2 = pd.concat([pred_2, pd.DataFrame(model.predict(x_test_2))], axis = 1)


result_1 = pred_2.iloc[:,0:9]
result_2 = pred_2.iloc[:,9:18]
result_3 = pred_2.iloc[:,18:27]
result_4 = pred_2.iloc[:,27:36]
result_5 = pred_2.iloc[:,36:45]

result = (result_1+result_2+result_3+result_4+result_5)/5
result.columns = sub.iloc[:,1:].columns

result

0
0.1
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 1.42945	valid_1's quantile: 1.42605
[1000]	training's quantile: 1.41769	valid_1's quantile: 1.42113
[1500]	training's quantile: 1.41138	valid_1's quantile: 1.41752
[2000]	training's quantile: 1.40401	valid_1's quantile: 1.41442
[2500]	training's quantile: 1.39865	valid_1's quantile: 1.41223
[3000]	training's quantile: 1.3919	valid_1's quantile: 1.41067
[3500]	training's quantile: 1.38782	valid_1's quantile: 1.40949
[4000]	training's quantile: 1.38574	valid_1's quantile: 1.40871
[4500]	training's quantile: 1.38424	valid_1's quantile: 1.40817
Early stopping, best iteration is:
[4642]	training's quantile: 1.38406	valid_1's quantile: 1.40807
0.2
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't 

[7500]	training's quantile: 2.04514	valid_1's quantile: 2.06402
[8000]	training's quantile: 2.04211	valid_1's quantile: 2.06166
[8500]	training's quantile: 2.03939	valid_1's quantile: 2.05991
[9000]	training's quantile: 2.03622	valid_1's quantile: 2.05861
[9500]	training's quantile: 2.0339	valid_1's quantile: 2.05758
[10000]	training's quantile: 2.03009	valid_1's quantile: 2.05427
Did not meet early stopping. Best iteration is:
[10000]	training's quantile: 2.03009	valid_1's quantile: 2.05427
0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 1.77735	valid_1's quantile: 1.78621
[1000]	training's quantile: 1.64867	valid_1's quantile: 1.65557
[1500]	training's quantile: 1.61807	valid_1's quantile: 1.6221
[2000]	training's quantile: 1.60095	valid_1's quantile: 1.60449
[2500]	training's quantile: 1.58506	valid_1's quantile: 1.58826
[3000]	trai

[10000]	training's quantile: 2.74027	valid_1's quantile: 2.84505
Did not meet early stopping. Best iteration is:
[10000]	training's quantile: 2.74027	valid_1's quantile: 2.84505
0.6
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.73649	valid_1's quantile: 2.77775
[1000]	training's quantile: 2.59618	valid_1's quantile: 2.64649
[1500]	training's quantile: 2.55997	valid_1's quantile: 2.61931
[2000]	training's quantile: 2.54233	valid_1's quantile: 2.60734
[2500]	training's quantile: 2.52485	valid_1's quantile: 2.59385
[3000]	training's quantile: 2.51317	valid_1's quantile: 2.58501
[3500]	training's quantile: 2.50389	valid_1's quantile: 2.58116
[4000]	training's quantile: 2.4949	valid_1's quantile: 2.57488
[4500]	training's quantile: 2.48775	valid_1's quantile: 2.56942
[5000]	training's quantile: 2.48004	valid_1's quantile: 2.56487
[5500]	tra

0.3
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.86161	valid_1's quantile: 2.87769
[1000]	training's quantile: 2.80958	valid_1's quantile: 2.83399
[1500]	training's quantile: 2.79433	valid_1's quantile: 2.82152
[2000]	training's quantile: 2.78151	valid_1's quantile: 2.81091
[2500]	training's quantile: 2.77137	valid_1's quantile: 2.80454
[3000]	training's quantile: 2.76088	valid_1's quantile: 2.79924
[3500]	training's quantile: 2.75501	valid_1's quantile: 2.79639
[4000]	training's quantile: 2.7511	valid_1's quantile: 2.79401
[4500]	training's quantile: 2.74724	valid_1's quantile: 2.7924
[5000]	training's quantile: 2.7415	valid_1's quantile: 2.78956
[5500]	training's quantile: 2.7354	valid_1's quantile: 2.78528
[6000]	training's quantile: 2.73083	valid_1's quantile: 2.78193
[6500]	training's quantile: 2.7279	valid_1's quantile: 2.78005


[2500]	training's quantile: 1.57408	valid_1's quantile: 1.62365
[3000]	training's quantile: 1.56283	valid_1's quantile: 1.61455
[3500]	training's quantile: 1.55363	valid_1's quantile: 1.60589
[4000]	training's quantile: 1.5455	valid_1's quantile: 1.59869
[4500]	training's quantile: 1.54004	valid_1's quantile: 1.59401
[5000]	training's quantile: 1.53653	valid_1's quantile: 1.59138
[5500]	training's quantile: 1.53296	valid_1's quantile: 1.58851
[6000]	training's quantile: 1.53017	valid_1's quantile: 1.58603
[6500]	training's quantile: 1.52731	valid_1's quantile: 1.58351
[7000]	training's quantile: 1.52499	valid_1's quantile: 1.5816
[7500]	training's quantile: 1.52276	valid_1's quantile: 1.57975
[8000]	training's quantile: 1.51843	valid_1's quantile: 1.5757
[8500]	training's quantile: 1.51623	valid_1's quantile: 1.57391
[9000]	training's quantile: 1.51444	valid_1's quantile: 1.57234
[9500]	training's quantile: 1.51253	valid_1's quantile: 1.57114
[10000]	training's quantile: 1.5096	valid_1

[6500]	training's quantile: 2.76519	valid_1's quantile: 2.81245
[7000]	training's quantile: 2.75958	valid_1's quantile: 2.81107
[7500]	training's quantile: 2.75501	valid_1's quantile: 2.80998
[8000]	training's quantile: 2.74921	valid_1's quantile: 2.80775
[8500]	training's quantile: 2.74532	valid_1's quantile: 2.80635
[9000]	training's quantile: 2.74217	valid_1's quantile: 2.80542
[9500]	training's quantile: 2.73939	valid_1's quantile: 2.80444
[10000]	training's quantile: 2.73603	valid_1's quantile: 2.80294
Did not meet early stopping. Best iteration is:
[10000]	training's quantile: 2.73603	valid_1's quantile: 2.80294
0.6
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 2.75221	valid_1's quantile: 2.70779
[1000]	training's quantile: 2.60954	valid_1's quantile: 2.58968
[1500]	training's quantile: 2.57153	valid_1's quantile: 2.56229
[2000]	tr

[3000]	training's quantile: 2.75296	valid_1's quantile: 2.83404
[3500]	training's quantile: 2.74718	valid_1's quantile: 2.83014
[4000]	training's quantile: 2.74053	valid_1's quantile: 2.827
[4500]	training's quantile: 2.73252	valid_1's quantile: 2.82492
[5000]	training's quantile: 2.7264	valid_1's quantile: 2.82174
[5500]	training's quantile: 2.71928	valid_1's quantile: 2.81959
[6000]	training's quantile: 2.71106	valid_1's quantile: 2.81633
[6500]	training's quantile: 2.70547	valid_1's quantile: 2.81494
Early stopping, best iteration is:
[6315]	training's quantile: 2.7064	valid_1's quantile: 2.81476
0.4
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Training until validation scores don't improve for 200 rounds
[500]	training's quantile: 3.03995	valid_1's quantile: 3.10729
[1000]	training's quantile: 2.96451	valid_1's quantile: 3.04116
[1500]	training's quantile: 2.9408	valid_1's quantile: 3.01977
[2000]	training's quantile: 2

[1000]	training's quantile: 0.967653	valid_1's quantile: 0.976246
[1500]	training's quantile: 0.937216	valid_1's quantile: 0.947614
[2000]	training's quantile: 0.922258	valid_1's quantile: 0.933944
[2500]	training's quantile: 0.912289	valid_1's quantile: 0.925368
[3000]	training's quantile: 0.904896	valid_1's quantile: 0.918243
[3500]	training's quantile: 0.898691	valid_1's quantile: 0.911938
[4000]	training's quantile: 0.892239	valid_1's quantile: 0.905482
[4500]	training's quantile: 0.884506	valid_1's quantile: 0.898476
[5000]	training's quantile: 0.878097	valid_1's quantile: 0.892996
[5500]	training's quantile: 0.872482	valid_1's quantile: 0.887863
[6000]	training's quantile: 0.868901	valid_1's quantile: 0.884546
[6500]	training's quantile: 0.865396	valid_1's quantile: 0.881369
[7000]	training's quantile: 0.861535	valid_1's quantile: 0.877499
[7500]	training's quantile: 0.85805	valid_1's quantile: 0.874162
[8000]	training's quantile: 0.855096	valid_1's quantile: 0.871769
[8500]	trai

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
1,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
2,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
3,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
4,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
3884,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
3885,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
3886,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33


In [24]:
sub.loc[sub.id.str.contains("Day8"), "q_0.1":] = result.sort_index().values

In [31]:
sub.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,3.520323,5.935774,7.679318,8.906169,9.834737,10.515792,11.014401,11.521004,12.122356
std,8.078073,13.176904,16.618778,18.997819,20.810194,22.092160,22.958609,23.753857,24.663374
min,-2.152752,-1.472993,-1.504698,-2.698070,-2.881432,-4.116466,-5.282867,-9.133564,-14.783810
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.288641,0.430270,0.790429,1.063804,1.104885,1.132686,1.778964,3.232109
max,47.703840,80.340962,93.864735,91.930091,97.047051,96.077966,96.980737,96.441509,97.921550


In [32]:
pred = sub.copy()

In [33]:
for i in pred.index:
    if pred['q_0.1'][i] < 0:
        pred['q_0.1'][i] = 0
for i in pred.index:
    if pred['q_0.2'][i] < 0:
        pred['q_0.2'][i] = 0
for i in pred.index:
    if pred['q_0.3'][i] < 0:
        pred['q_0.3'][i] = 0
for i in pred.index:
    if pred['q_0.4'][i] < 0:
        pred['q_0.4'][i] = 0
for i in pred.index:
    if pred['q_0.5'][i] < 0:
        pred['q_0.5'][i] = 0
for i in pred.index:
    if pred['q_0.6'][i] < 0:
        pred['q_0.6'][i] = 0
for i in pred.index:
    if pred['q_0.7'][i] < 0:
        pred['q_0.7'][i] = 0
for i in pred.index:
    if pred['q_0.8'][i] < 0:
        pred['q_0.8'][i] = 0
for i in pred.index:
    if pred['q_0.9'][i] < 0:
        pred['q_0.9'][i] = 0
    
pred

<ipython-input-33-9f77b97f5850>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['q_0.1'][i] = 0
<ipython-input-33-9f77b97f5850>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['q_0.2'][i] = 0
<ipython-input-33-9f77b97f5850>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['q_0.3'][i] = 0
<ipython-input-33-9f77b97f5850>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,1.105373e-33,2.900734e-25,1.106410e-33,1.106345e-33


In [34]:
pred.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,3.523633,5.937689,7.682659,8.908821,9.837480,10.522082,11.023492,11.540718,12.157866
std,8.076470,13.175998,16.617134,18.996492,20.808817,22.088853,22.953691,23.741830,24.639522
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.288641,0.430270,0.790429,1.063804,1.104885,1.132686,1.778964,3.232109
max,47.703840,80.340962,93.864735,91.930091,97.047051,96.077966,96.980737,96.441509,97.921550


In [40]:
pred_lgbm = pred.copy()

In [80]:
from keras.layers import Activation, Dense
from keras.models import Sequential
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.utils import get_custom_objects
import tensorflow.keras.backend as K
from keras.layers import Dropout
from keras.layers import BatchNormalization

def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

class Mish(Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'

def mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': Mish(mish)})

In [81]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_train_1_scaler = scaler.fit_transform(x_train_1)
x_test_1_scaler = scaler.transform(x_test_1)

x_train_2_scaler = scaler.fit_transform(x_train_2)
x_test_2_scaler = scaler.transform(x_test_2)

In [82]:
model = Sequential()
model.add(Dense(7, input_dim=7))
model.add(Activation('relu'))
model.add(Dense(14))
model.add(Activation('relu'))
model.add(Dense(7))
model.add(Activation('relu'))
model.add(Dense(1, activation='linear'))

In [83]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

In [84]:
x_train_1_scaler = pd.DataFrame(x_train_1_scaler, columns = x_train_1.columns)
x_train_2_scaler = pd.DataFrame(x_train_2_scaler, columns = x_train_2.columns)
x_test_1_scaler = pd.DataFrame(x_test_1_scaler, columns = x_test_1.columns)
x_test_2_scaler = pd.DataFrame(x_test_2_scaler, columns = x_test_2.columns)

In [85]:
quantile_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
pred_1 = pd.DataFrame()
for n_fold, (train_idx, val_idx) in enumerate(folds.split(x_train_1_scaler)):
    train_x, train_y = x_train_1_scaler.loc[train_idx], y_train_1.loc[train_idx]
    val_x, val_y = x_train_1_scaler.loc[val_idx], y_train_1.loc[val_idx]
    print(n_fold)
    for j in quantile_list:
        print(j)
        quantile = j
        model.compile(loss=lambda y,f: tilted_loss(quantile,y,f), optimizer='adam')
        model.fit(train_x, train_y, validation_data = (val_x, val_y), epochs=500,  batch_size=128, callbacks=[es])
        pred_1 = pd.concat([pred_1, pd.DataFrame(model.predict(x_test_1_scaler))], axis = 1)
        
result_1 = pred_1.iloc[:,0:9]
result_2 = pred_1.iloc[:,9:18]
result_3 = pred_1.iloc[:,18:27]
result_4 = pred_1.iloc[:,27:36]
result_5 = pred_1.iloc[:,36:45]

result = (result_1+result_2+result_3+result_4+result_5)/5
result.columns = sub.iloc[:,1:].columns

result

0
0.1
Epoch 1/500
329/329 [==============================] - 4s 4ms/step - loss: 1.7612 - val_loss: 1.4443
Epoch 2/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4650 - val_loss: 1.4325
Epoch 3/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4576 - val_loss: 1.4318
Epoch 4/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4325 - val_loss: 1.4233
Epoch 5/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4523 - val_loss: 1.4102
Epoch 6/500
329/329 [==============================] - 1s 3ms/step - loss: 1.4610 - val_loss: 1.4011
Epoch 7/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4294 - val_loss: 1.4018
Epoch 8/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4192 - val_loss: 1.3901
Epoch 9/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4219 - val_loss: 1.3881
Epoch 10/500
329/329 [==============================] - 1s 2ms/step - loss: 1.4273 - 

Epoch 20/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2059 - val_loss: 2.1675
Epoch 21/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1858 - val_loss: 2.1775
Epoch 22/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1522 - val_loss: 2.1752
Epoch 23/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1987 - val_loss: 2.1699
Epoch 24/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1818 - val_loss: 2.1810
Epoch 25/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2250 - val_loss: 2.1718
Epoch 26/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1720 - val_loss: 2.1702
Epoch 27/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1617 - val_loss: 2.1699
Epoch 28/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1733 - val_loss: 2.1751
Epoch 29/500
329/329 [==============================] - 1s 2ms/step - loss: 2.1924

329/329 [==============================] - 1s 2ms/step - loss: 2.5820 - val_loss: 2.6029
Epoch 32/500
329/329 [==============================] - 1s 2ms/step - loss: 2.6271 - val_loss: 2.6154
Epoch 33/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5855 - val_loss: 2.6136
Epoch 34/500
329/329 [==============================] - 1s 2ms/step - loss: 2.6074 - val_loss: 2.6156
Epoch 35/500
329/329 [==============================] - 1s 2ms/step - loss: 2.6037 - val_loss: 2.6138
Epoch 36/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5911 - val_loss: 2.6132
Epoch 37/500
329/329 [==============================] - 1s 2ms/step - loss: 2.6258 - val_loss: 2.6191
Epoch 38/500
329/329 [==============================] - 1s 2ms/step - loss: 2.6353 - val_loss: 2.6020
Epoch 39/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5964 - val_loss: 2.6168
Epoch 40/500
329/329 [==============================] - 1s 2ms/step - loss: 2.6043 - val_loss: 

Epoch 24/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2655 - val_loss: 2.3179
Epoch 25/500
329/329 [==============================] - 1s 2ms/step - loss: 2.3667 - val_loss: 2.3261
Epoch 26/500
329/329 [==============================] - 1s 2ms/step - loss: 2.3251 - val_loss: 2.3151
Epoch 27/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2978 - val_loss: 2.3374
Epoch 28/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2978 - val_loss: 2.3229
Epoch 29/500
329/329 [==============================] - 1s 2ms/step - loss: 2.3346 - val_loss: 2.3178
Epoch 30/500
329/329 [==============================] - 1s 2ms/step - loss: 2.3422 - val_loss: 2.3408
Epoch 31/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2616 - val_loss: 2.3161
Epoch 32/500
329/329 [==============================] - 1s 2ms/step - loss: 2.2906 - val_loss: 2.3212
Epoch 33/500
329/329 [==============================] - 1s 2ms/step - loss: 2.3046

329/329 [==============================] - 1s 2ms/step - loss: 1.9089 - val_loss: 1.8959
Epoch 64/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9215 - val_loss: 1.8991
Epoch 65/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9042 - val_loss: 1.9159
Epoch 66/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9231 - val_loss: 1.9077
Epoch 67/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9318 - val_loss: 1.8915
Epoch 68/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9147 - val_loss: 1.9058
Epoch 69/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9543 - val_loss: 1.9087
Epoch 70/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9424 - val_loss: 1.8977
Epoch 71/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9291 - val_loss: 1.9053
Epoch 72/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9207 - val_loss: 

...

329/329 [==============================] - 1s 2ms/step - loss: 1.3568 - val_loss: 1.3974
Epoch 40/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3562 - val_loss: 1.3946
Epoch 41/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3614 - val_loss: 1.3959
Epoch 42/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3794 - val_loss: 1.4032
Epoch 43/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3662 - val_loss: 1.3939
Epoch 44/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3603 - val_loss: 1.3994
Epoch 45/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3512 - val_loss: 1.3973
Epoch 46/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3562 - val_loss: 1.3943
Epoch 47/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3566 - val_loss: 1.4007
Epoch 48/500
329/329 [==============================] - 1s 2ms/step - loss: 1.3466 - val_loss: 

329/329 [==============================] - 1s 2ms/step - loss: 2.5852 - val_loss: 2.6463
Epoch 82/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5892 - val_loss: 2.6611
Epoch 83/500
329/329 [==============================] - ETA: 0s - loss: 2.557 - 1s 2ms/step - loss: 2.5579 - val_loss: 2.6488
Epoch 84/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5621 - val_loss: 2.6553
Epoch 85/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5827 - val_loss: 2.6532
Epoch 86/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5837 - val_loss: 2.6602
Epoch 87/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5462 - val_loss: 2.6536
Epoch 88/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5620 - val_loss: 2.6663
Epoch 89/500
329/329 [==============================] - 1s 2ms/step - loss: 2.5311 - val_loss: 2.6498
Epoch 90/500
329/329 [==============================] - 1s 2ms/step - l

Epoch 16/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8699 - val_loss: 1.9259
Epoch 17/500
329/329 [==============================] - 1s 2ms/step - loss: 1.9050 - val_loss: 1.9175
Epoch 18/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8886 - val_loss: 1.9216
Epoch 19/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8516 - val_loss: 1.9191
Epoch 20/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8980 - val_loss: 1.9389
Epoch 21/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8575 - val_loss: 1.9138
Epoch 22/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8736 - val_loss: 1.9480
Epoch 23/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8716 - val_loss: 1.9161
Epoch 24/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8664 - val_loss: 1.9223
Epoch 25/500
329/329 [==============================] - 1s 2ms/step - loss: 1.8878

Epoch 27/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7502 - val_loss: 0.7775
Epoch 28/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7598 - val_loss: 0.7898
Epoch 29/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7520 - val_loss: 0.7781
Epoch 30/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7390 - val_loss: 0.7811
Epoch 31/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7585 - val_loss: 0.7886
Epoch 32/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7527 - val_loss: 0.7876
Epoch 33/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7606 - val_loss: 0.7845
Epoch 34/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7613 - val_loss: 0.7825
Epoch 35/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7476 - val_loss: 0.7812
Epoch 36/500
329/329 [==============================] - 1s 2ms/step - loss: 0.7587

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
1,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
2,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
3,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
4,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
...,...,...,...,...,...,...,...,...,...
3883,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
3884,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
3885,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273
3886,-0.00027,-0.000271,-0.000211,-0.000109,0.000061,0.000144,0.000296,0.000191,0.000273


In [86]:
sub.loc[sub.id.str.contains("Day7"), "q_0.1":] = result.sort_index().values

In [87]:
quantile_list = [0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85]
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)
pred_2 = pd.DataFrame()
for n_fold, (train_idx, val_idx) in enumerate(folds.split(x_train_2_scaler)):
    train_x, train_y = x_train_2_scaler.loc[train_idx], y_train_2.loc[train_idx]
    val_x, val_y = x_train_2_scaler.loc[val_idx], y_train_2.loc[val_idx]
    print(n_fold)
    for j in quantile_list:
        print(j)
        quantile = j
        model.compile(loss=lambda y,f: tilted_loss(quantile,y,f), optimizer='adam')
        model.fit(train_x, train_y, validation_data = (val_x, val_y), epochs=500,  batch_size=128, callbacks=[es])
        pred_2 = pd.concat([pred_2, pd.DataFrame(model.predict(x_test_2_scaler))], axis = 1)
        
result_1 = pred_2.iloc[:,0:9]
result_2 = pred_2.iloc[:,9:18]
result_3 = pred_2.iloc[:,18:27]
result_4 = pred_2.iloc[:,27:36]
result_5 = pred_2.iloc[:,36:45]

result = (result_1+result_2+result_3+result_4+result_5)/5
result.columns = sub.iloc[:,1:].columns

result

0
0.05
Epoch 1/500
328/328 [==============================] - 2s 3ms/step - loss: 2.1027 - val_loss: 0.7901
Epoch 2/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7815 - val_loss: 0.7829
Epoch 3/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7868 - val_loss: 0.7769
Epoch 4/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7777 - val_loss: 0.7760
Epoch 5/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7938 - val_loss: 0.7725
Epoch 6/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7759 - val_loss: 0.7787
Epoch 7/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7775 - val_loss: 0.7727
Epoch 8/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7853 - val_loss: 0.7706
Epoch 9/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7863 - val_loss: 0.7721
Epoch 10/500
328/328 [==============================] - 1s 2ms/step - loss: 0.7753 -

Epoch 18/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9102 - val_loss: 1.9191
Epoch 19/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9193 - val_loss: 1.9164
Epoch 20/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9033 - val_loss: 1.9124
Epoch 21/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9178 - val_loss: 1.9147
Epoch 22/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9110 - val_loss: 1.9206
Epoch 23/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8771 - val_loss: 1.9188
Epoch 24/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9239 - val_loss: 1.9183
Epoch 25/500
328/328 [==============================] - 1s 2ms/step - loss: 1.9078 - val_loss: 1.9226
Epoch 26/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8778 - val_loss: 1.9125
Epoch 27/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8747

Epoch 24/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8551 - val_loss: 2.8724
Epoch 25/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8512 - val_loss: 2.8560
Epoch 26/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8613 - val_loss: 2.8544
Epoch 27/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8086 - val_loss: 2.8645
Epoch 28/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8232 - val_loss: 2.8447
Epoch 29/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8188 - val_loss: 2.8567
Epoch 30/500
328/328 [==============================] - 1s 2ms/step - loss: 2.7921 - val_loss: 2.8503
Epoch 31/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8173 - val_loss: 2.8502
Epoch 32/500
328/328 [==============================] - 1s 2ms/step - loss: 2.7683 - val_loss: 2.8558
Epoch 33/500
328/328 [==============================] - 1s 2ms/step - loss: 2.8285

Epoch 8/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5431 - val_loss: 2.5848
Epoch 9/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5498 - val_loss: 2.5778
Epoch 10/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5995 - val_loss: 2.5799
Epoch 11/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5433 - val_loss: 2.5774
Epoch 12/500
328/328 [==============================] - 1s 2ms/step - loss: 2.6040 - val_loss: 2.5750
Epoch 13/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5921 - val_loss: 2.5781
Epoch 14/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5963 - val_loss: 2.5763
Epoch 15/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5603 - val_loss: 2.5733
Epoch 16/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5807 - val_loss: 2.5836
Epoch 17/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5456 -

328/328 [==============================] - 1s 2ms/step - loss: 2.2141 - val_loss: 2.2005
Epoch 48/500
328/328 [==============================] - 1s 2ms/step - loss: 2.2426 - val_loss: 2.2003
Epoch 49/500
328/328 [==============================] - 1s 2ms/step - loss: 2.1923 - val_loss: 2.1969
Epoch 50/500
328/328 [==============================] - 1s 2ms/step - loss: 2.1850 - val_loss: 2.2107
Epoch 51/500
328/328 [==============================] - 1s 2ms/step - loss: 2.1674 - val_loss: 2.2106
Epoch 52/500
328/328 [==============================] - 1s 2ms/step - loss: 2.2286 - val_loss: 2.2062
Epoch 53/500
328/328 [==============================] - 1s 2ms/step - loss: 2.2184 - val_loss: 2.1964
Epoch 54/500
328/328 [==============================] - 1s 2ms/step - loss: 2.1779 - val_loss: 2.1910
Epoch 55/500
328/328 [==============================] - 1s 2ms/step - loss: 2.2256 - val_loss: 2.1935
Epoch 56/500
328/328 [==============================] - 1s 2ms/step - loss: 2.1813 - val_loss: 

...

Epoch 25/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8688 - val_loss: 1.9202
Epoch 26/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8670 - val_loss: 1.9165
Epoch 27/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8998 - val_loss: 1.9143
Epoch 28/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8731 - val_loss: 1.9032
Epoch 29/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8936 - val_loss: 1.9184
Epoch 30/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8705 - val_loss: 1.9097
Epoch 31/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8489 - val_loss: 1.9101
Epoch 32/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8485 - val_loss: 1.9122
Epoch 33/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8763 - val_loss: 1.9099
Epoch 34/500
328/328 [==============================] - 1s 2ms/step - loss: 1.8824

Epoch 24/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5457 - val_loss: 2.5847
Epoch 25/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5812 - val_loss: 2.5760
Epoch 26/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5616 - val_loss: 2.5754
Epoch 27/500
328/328 [==============================] - 1s 2ms/step - loss: 2.6252 - val_loss: 2.5848
Epoch 28/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5679 - val_loss: 2.5763
Epoch 29/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5898 - val_loss: 2.5785
Epoch 30/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5894 - val_loss: 2.5824
Epoch 31/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5733 - val_loss: 2.5771
Epoch 32/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5905 - val_loss: 2.5800
Epoch 33/500
328/328 [==============================] - 1s 2ms/step - loss: 2.5764

Epoch 16/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7167 - val_loss: 1.7250
Epoch 17/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7018 - val_loss: 1.7085
Epoch 18/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7205 - val_loss: 1.7109
Epoch 19/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7185 - val_loss: 1.7071
Epoch 20/500
328/328 [==============================] - 1s 2ms/step - loss: 1.6817 - val_loss: 1.7103
Epoch 21/500
328/328 [==============================] - 1s 2ms/step - loss: 1.6887 - val_loss: 1.7105
Epoch 22/500
328/328 [==============================] - 1s 2ms/step - loss: 1.6796 - val_loss: 1.7128
Epoch 23/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7059 - val_loss: 1.7069
Epoch 24/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7073 - val_loss: 1.7211
Epoch 25/500
328/328 [==============================] - 1s 2ms/step - loss: 1.7050

328/328 [==============================] - 1s 2ms/step - loss: 1.1102 - val_loss: 1.1169
Epoch 68/500
328/328 [==============================] - 1s 2ms/step - loss: 1.1081 - val_loss: 1.1208
Epoch 69/500
328/328 [==============================] - 1s 2ms/step - loss: 1.0977 - val_loss: 1.1175
Epoch 70/500
328/328 [==============================] - 1s 2ms/step - loss: 1.1210 - val_loss: 1.1106
Epoch 71/500
328/328 [==============================] - 1s 2ms/step - loss: 1.0937 - val_loss: 1.1282
Epoch 72/500
328/328 [==============================] - 1s 2ms/step - loss: 1.0890 - val_loss: 1.1132
Epoch 73/500
328/328 [==============================] - 1s 2ms/step - loss: 1.1063 - val_loss: 1.1149
Epoch 74/500
328/328 [==============================] - 1s 2ms/step - loss: 1.1227 - val_loss: 1.1128
Epoch 00074: early stopping


,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
1,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
2,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
3,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
4,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
...,...,...,...,...,...,...,...,...,...
3883,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
3884,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
3885,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321
3886,-0.00082,-0.000278,-0.000117,-0.000086,-0.000142,0.000019,0.000097,0.000237,0.000321


In [88]:
sub.loc[sub.id.str.contains("Day8"), "q_0.1":] = result.sort_index().values

In [89]:
sub.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,5.642376,10.690382,14.508972,17.311780,19.353778,20.858315,21.712103,22.673416,23.360904
std,9.075053,15.507614,20.047825,23.376371,25.763067,27.504193,28.457959,29.446399,30.083211
min,-0.000820,-0.000278,-0.000211,-0.000109,-0.000142,0.000019,0.000097,0.000191,0.000273
25%,-0.000820,-0.000278,-0.000117,-0.000086,0.000061,0.000144,0.000296,0.000237,0.000321
50%,-0.000270,-0.000271,-0.000117,0.071692,0.250250,0.588568,0.630258,1.113779,1.541545
75%,8.804232,19.122424,27.335909,33.868988,37.920055,41.390615,43.738308,45.976662,47.003334
max,47.147179,68.184662,77.626648,86.494949,90.500992,94.835655,95.420494,97.376747,98.477737


In [90]:
sub = sub.round(2)

In [91]:
sub.iloc[:,1:] = abs(sub.iloc[:,1:])

In [94]:
pred.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,5.642672,10.690472,14.509029,17.311816,19.353831,20.858302,21.712028,22.673332,23.360808
std,9.074915,15.507473,20.047747,23.376328,25.763055,27.504275,28.458049,29.446488,30.083371
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.075000,0.250000,0.590000,0.630000,1.115000,1.545000
75%,8.802500,19.120000,27.332500,33.870000,37.925000,41.392500,43.740000,45.975000,47.002500
max,47.150000,68.180000,77.630000,86.490000,90.500000,94.840000,95.420000,97.380000,98.480000


In [95]:
pred.to_csv('0126_second.csv', index = False)

In [92]:
pred = sub.copy()

In [93]:
for i in pred.index:
    if pred['q_0.1'][i] < 0:
        pred['q_0.1'][i] = 0
for i in pred.index:
    if pred['q_0.2'][i] < 0:
        pred['q_0.2'][i] = 0
for i in pred.index:
    if pred['q_0.3'][i] < 0:
        pred['q_0.3'][i] = 0
for i in pred.index:
    if pred['q_0.4'][i] < 0:
        pred['q_0.4'][i] = 0
for i in pred.index:
    if pred['q_0.5'][i] < 0:
        pred['q_0.5'][i] = 0
for i in pred.index:
    if pred['q_0.6'][i] < 0:
        pred['q_0.6'][i] = 0
for i in pred.index:
    if pred['q_0.7'][i] < 0:
        pred['q_0.7'][i] = 0
for i in pred.index:
    if pred['q_0.8'][i] < 0:
        pred['q_0.8'][i] = 0
for i in pred.index:
    if pred['q_0.9'][i] < 0:
        pred['q_0.9'][i] = 0
    
pred

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
pred_neu = pred.copy()

In [56]:
sub

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
sub.iloc[:,1:] = ((pred_lgbm.iloc[:,1:] + pred_neu.iloc[:,1:])/2).round(2)

In [60]:
pred_neu.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,6.112645,10.809464,14.131773,17.161394,18.947621,20.499542,22.147126,23.016097,24.472436
std,9.740513,15.586740,19.482012,23.101532,25.166958,27.000063,28.946973,29.845326,31.362877
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.030000,0.120000,0.130000,0.290000,0.565000,1.140000
75%,10.232500,19.820000,27.355000,34.202500,38.050000,41.540000,45.335000,47.145000,50.002500
max,49.790000,69.790000,79.270000,87.570000,91.610000,94.380000,99.010000,98.540000,103.330000
